In [1]:
import numpy as np
import pandas as pd
from skimage import io, transform
import os
import sklearn.preprocessing
import sys
import random
import tensorflow as tf
import tensorflow.contrib as tf_contrib
import time
import _pickle as pickle

D:\Python\anaconda3.4\envs\py36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# 读取训练图片列表
all_data = pd.read_csv('data/label.csv')

# 读取测试图片列表
test_data_list = pd.read_csv('data/test.csv')

#全局one-hot编码空间
label_binarizer = ""

#初始的定义的数据
class_num = 12
image_size = 32
img_channels = 3
iterations = 72
batch_size = 24
total_epoch = 100
weight_decay = 0.0003
dropout_rate = 0.5
momentum_rate = 0.9
log_save_path = './vgg_19_logs'
model_save_path = './model/'
batch_size_test = 4

In [3]:
#默认按列归一化
def def_normalization(x,h = 1):
    # 记录归一化全局最大值最小值，回代时需要
    if h==1:
        #按列归一化
        amin, amax = np.min(x, 0), np.max(x, 0)
        xx = (x - amin) / (amax - amin)
    else:
        #按行归一化
        amin, amax = np.min(x, 1), np.max(x, 1)
        xx = ((x.T - amin) / (amax - amin)).T
    #记录归一化最大值最小值，回代时需要
    return xx

#使用one-hot编码，将离散特征的取值扩展到了欧式空间
def def_one_hot(x):
    if label_binarizer == "":
        binarizer = sklearn.preprocessing.LabelBinarizer()
    else:
        binarizer = label_binarizer
    binarizer.fit(x)
    y= binarizer.transform(x)
    return y

In [4]:
# 读取的图片，顺序打乱，划分测试和训练
def read_new_img(img_path,label):
    imgs=[]
    labels = label
    for idx in img_path:
        img = io.imread(idx)
        img = transform.resize(img, (image_size, image_size))
        imgs.append(img)
        print(idx + '------------is ok ')
    x_data, x_label = np.array(imgs), np.array(def_one_hot(np.array(labels)))
    data = []
    sigle_data = []
    for i in range(len(x_data)):
        sigle_data.append(x_data[i])
        sigle_data.append(x_label[i])
        data.append(sigle_data)
        sigle_data = []
        # 打乱顺序
    data = np.array(data)
    num_example = data.shape[0]
    np.random.shuffle(data)
    # 将所有数据分为训练集和验证集
    ratio = 0.8
    img_train = []
    label_train = []
    img_test = []
    label_test = []
    for i in range(int(ratio*num_example)):
        img_train.append(data[i][0])
        label_train.append(data[i][1])
    img_train = np.array(img_train)
    label_train = np.array(label_train)
    for i in range(int(ratio*num_example),num_example):
        img_test.append(data[i][0])
        label_test.append(data[i][1])
    img_test = np.array(img_test)
    label_test = np.array(label_test)
    print(img_train)
    return img_train,img_test,label_train,label_test

In [5]:
#加载数据
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo,encoding='latin1')
    return dict

def load_data_one(file):
    batch = unpickle(file)
    data = batch['data']
    labels = batch['labels']
    print("Loading %s : %d." % (file, len(data)))
    return data, labels

def load_data(files, data_dir, label_count):
    global image_size, img_channels
    data, labels = load_data_one(data_dir + '/' + files[0])
    for f in files[1:]:
        data_n, labels_n = load_data_one(data_dir + '/' + f)
        data = np.append(data, data_n, axis=0)
        labels = np.append(labels, labels_n, axis=0)
    labels = np.array([[float(i == label) for i in range(label_count)] for label in labels])
    data = data.reshape([-1, img_channels, image_size, image_size])
    print('dataShape is ----------------:')
    print(data.shape)
    data = data.transpose([0, 2, 3, 1])
    return data, labels


def prepare_data():
    print("======Loading data======")
    data_dir = './cifar-10-python/cifar-10-batches-py'
    image_dim = image_size * image_size * img_channels
    meta = unpickle(data_dir + '/batches.meta')

    label_names = meta['label_names']
    label_count = len(label_names)
    train_files = ['data_batch_%d' % d for d in range(1, 6)]
    train_data, train_labels = load_data(train_files, data_dir, label_count)
    test_data, test_labels = load_data(['test_batch'], data_dir, label_count)

    print("Train data:", np.shape(train_data), np.shape(train_labels))
    print("Test data :", np.shape(test_data), np.shape(test_labels))
    print("======Load finished======")

    print("======Shuffling data======")
    indices = np.random.permutation(len(train_data))
    train_data = train_data[indices]
    train_labels = train_labels[indices]
    print("======Prepare Finished======")

    return train_data, train_labels, test_data, test_labels

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape, dtype=tf.float32)
    return tf.Variable(initial)


def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')


def max_pool(input, k_size=1, stride=1, name=None):
    return tf.nn.max_pool(input, ksize=[1, k_size, k_size, 1], strides=[1, stride, stride, 1],
                          padding='SAME', name=name)


def batch_norm(input):
    return tf.contrib.layers.batch_norm(input, decay=0.9, center=True, scale=True, epsilon=1e-3,
                                        is_training=train_flag, updates_collections=None)


def instance_norm(x, scope='instance_norm'):
    return tf_contrib.layers.instance_norm(x,
                                           epsilon=1e-05,
                                           center=True, scale=True,
                                           scope=scope)

def layer_norm(x, scope='layer_norm') :
    return tf_contrib.layers.layer_norm(x,
                                        center=True, scale=True,
                                        scope=scope)

def norm(x, norm_type, is_train,i, G=32, esp=1e-5):
    with tf.variable_scope('{}_norm_{}'.format(norm_type,i)):
        if norm_type == 'none':
            output = x
        elif norm_type == 'batch':
            output = tf.contrib.layers.batch_norm(
                x, center=True, scale=True, decay=0.999,
                is_training=is_train, updates_collections=None
            )
        elif norm_type == 'group':
            # normalize
            # tranpose: [bs, h, w, c] to [bs, c, h, w] following the paper
            x = tf.transpose(x, [0, 3, 1, 2])
            N, C, H, W = x.get_shape().as_list()
            G = min(G, C)
            x = tf.reshape(x, [-1, G, C // G, H, W])   # <------------------------------这里源码错了 需要改成这样 https://github.com/shaohua0116/Group-Normalization-Tensorflow/issues/1
            mean, var = tf.nn.moments(x, [2, 3, 4], keep_dims=True)
            x = (x - mean) / tf.sqrt(var + esp)
            # per channel gamma and beta
            gamma = tf.get_variable('gamma', [C],
                                    initializer=tf.constant_initializer(1.0))
            beta = tf.get_variable('beta', [C],
                                   initializer=tf.constant_initializer(0.0))
            gamma = tf.reshape(gamma, [1, C, 1, 1])
            beta = tf.reshape(beta, [1, C, 1, 1])

            output = tf.reshape(x, [-1, C, H, W]) * gamma + beta   # 《------------------------ 这里源码错了 需要改成这样 https://github.com/shaohua0116/Group-Normalization-Tensorflow/issues/1
            # tranpose: [bs, c, h, w, c] to [bs, h, w, c] following the paper
            output = tf.transpose(output, [0, 2, 3, 1])
        else:
            raise NotImplementedError
    return output


def _random_crop(batch, crop_shape, padding=None):
    oshape = np.shape(batch[0])

    if padding:
        oshape = (oshape[0] + 2*padding, oshape[1] + 2*padding)
    new_batch = []
    npad = ((padding, padding), (padding, padding), (0, 0))
    for i in range(len(batch)):
        new_batch.append(batch[i])
        if padding:
            new_batch[i] = np.lib.pad(batch[i], pad_width=npad,
                                      mode='constant', constant_values=0)
        nh = random.randint(0, oshape[0] - crop_shape[0])
        nw = random.randint(0, oshape[1] - crop_shape[1])
        new_batch[i] = new_batch[i][nh:nh + crop_shape[0],
                                    nw:nw + crop_shape[1]]
    return new_batch


def _random_flip_leftright(batch):
        for i in range(len(batch)):
            if bool(random.getrandbits(1)):
                batch[i] = np.fliplr(batch[i])
        return batch


def data_preprocessing(x_train,x_test):

    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')

    x_train[:, :, :, 0] = (x_train[:, :, :, 0] - np.mean(x_train[:, :, :, 0])) / np.std(x_train[:, :, :, 0])
    x_train[:, :, :, 1] = (x_train[:, :, :, 1] - np.mean(x_train[:, :, :, 1])) / np.std(x_train[:, :, :, 1])
    x_train[:, :, :, 2] = (x_train[:, :, :, 2] - np.mean(x_train[:, :, :, 2])) / np.std(x_train[:, :, :, 2])

    x_test[:, :, :, 0] = (x_test[:, :, :, 0] - np.mean(x_test[:, :, :, 0])) / np.std(x_test[:, :, :, 0])
    x_test[:, :, :, 1] = (x_test[:, :, :, 1] - np.mean(x_test[:, :, :, 1])) / np.std(x_test[:, :, :, 1])
    x_test[:, :, :, 2] = (x_test[:, :, :, 2] - np.mean(x_test[:, :, :, 2])) / np.std(x_test[:, :, :, 2])

    return x_train, x_test


def data_augmentation(batch):
    batch = _random_flip_leftright(batch)
    batch = _random_crop(batch, [32, 32], 4)
    return batch


def learning_rate_schedule(epoch_num):
    if epoch_num < 81:
        return 0.1
    elif epoch_num < 121:
        return 0.01
    else:
        return 0.001


def run_testing(sess, ep):
    acc = 0.0
    loss = 0.0
    pre_index = 0
    add = 40
    for it in range(10):
        batch_x = test_x[pre_index:pre_index+add]
        batch_y = test_y[pre_index:pre_index+add]
        pre_index = pre_index + add
        loss_, acc_  = sess.run([cross_entropy, accuracy],
                                feed_dict={x: batch_x, y_: batch_y, keep_prob: 1.0, train_flag: False})
        loss += loss_ / 10.0
        acc += acc_ / 10.0
    summary = tf.Summary(value=[tf.Summary.Value(tag="test_loss", simple_value=loss), 
                                tf.Summary.Value(tag="test_accuracy", simple_value=acc)])
    return acc, loss, summary


In [ ]:
if __name__ == '__main__':
    
    train_x, test_x, train_y, test_y = read_new_img(all_data['img_path'],all_data['label'])
    train_x, test_x = data_preprocessing(train_x, test_x)
     # define placeholder x, y_ , keep_prob, learning_rate
    x = tf.placeholder(tf.float32,[None, image_size, image_size, 3])
    y_ = tf.placeholder(tf.float32, [None, class_num])
    keep_prob = tf.placeholder(tf.float32)
    learning_rate = tf.placeholder(tf.float32)
    train_flag = tf.placeholder(tf.bool)

    # build_network

    W_conv1_1 = tf.get_variable('conv1_1', shape=[3, 3, 3, 64], initializer=tf.contrib.keras.initializers.he_normal())
    b_conv1_1 = bias_variable([64])
    output = tf.nn.relu(norm(conv2d(x, W_conv1_1) + b_conv1_1,norm_type='group',is_train = True,i='1_1'))
    print(output)
    
    W_conv1_2 = tf.get_variable('conv1_2', shape=[3, 3, 64, 64], initializer=tf.contrib.keras.initializers.he_normal())
    b_conv1_2 = bias_variable([64])
    output = tf.nn.relu(norm(conv2d(output, W_conv1_2) + b_conv1_2,norm_type='group',is_train = True,i='1_2'))
    output = max_pool(output, 2, 2, "pool1")
    print(output)

    W_conv2_1 = tf.get_variable('conv2_1', shape=[3, 3, 64, 128], initializer=tf.contrib.keras.initializers.he_normal())
    b_conv2_1 = bias_variable([128])
    output = tf.nn.relu(norm(conv2d(output, W_conv2_1) + b_conv2_1,norm_type='group',is_train = True,i='2_1'))
    print(output)

    W_conv2_2 = tf.get_variable('conv2_2', shape=[3, 3, 128, 128], initializer=tf.contrib.keras.initializers.he_normal())
    b_conv2_2 = bias_variable([128])
    output = tf.nn.relu(norm(conv2d(output, W_conv2_2) + b_conv2_2,norm_type='group',is_train = True,i='2_2'))
    output = max_pool(output, 2, 2, "pool2")
    print(output)

    W_conv3_1 = tf.get_variable('conv3_1', shape=[3, 3, 128, 256], initializer=tf.contrib.keras.initializers.he_normal())
    b_conv3_1 = bias_variable([256])
    output = tf.nn.relu( norm(conv2d(output,W_conv3_1) + b_conv3_1,norm_type='group',is_train = True,i='3_1'))
    print(output)

    W_conv3_2 = tf.get_variable('conv3_2', shape=[3, 3, 256, 256], initializer=tf.contrib.keras.initializers.he_normal())
    b_conv3_2 = bias_variable([256])
    output = tf.nn.relu(norm(conv2d(output, W_conv3_2) + b_conv3_2,norm_type='group',is_train = True,i='3_2'))
    print(output)

    W_conv3_3 = tf.get_variable('conv3_3', shape=[3, 3, 256, 256], initializer=tf.contrib.keras.initializers.he_normal())
    b_conv3_3 = bias_variable([256])
    output = tf.nn.relu( norm(conv2d(output, W_conv3_3) + b_conv3_3,norm_type='group',is_train = True,i='3_3'))
    print(output)

    W_conv3_4 = tf.get_variable('conv3_4', shape=[3, 3, 256, 256], initializer=tf.contrib.keras.initializers.he_normal())
    b_conv3_4 = bias_variable([256])
    output = tf.nn.relu(norm(conv2d(output, W_conv3_4) + b_conv3_4,norm_type='group',is_train = True,i='3_4'))
    output = max_pool(output, 2, 2, "pool3")
    print(output)

    W_conv4_1 = tf.get_variable('conv4_1', shape=[3, 3, 256, 512], initializer=tf.contrib.keras.initializers.he_normal())
    b_conv4_1 = bias_variable([512])
    output = tf.nn.relu(norm(conv2d(output, W_conv4_1) + b_conv4_1,norm_type='group',is_train = True,i='4_1'))
    print(output)

    W_conv4_2 = tf.get_variable('conv4_2', shape=[3, 3, 512, 512], initializer=tf.contrib.keras.initializers.he_normal())
    b_conv4_2 = bias_variable([512])
    output = tf.nn.relu(norm(conv2d(output, W_conv4_2) + b_conv4_2,norm_type='group',is_train = True,i='4_2'))
    print(output)

    W_conv4_3 = tf.get_variable('conv4_3', shape=[3, 3, 512, 512], initializer=tf.contrib.keras.initializers.he_normal())
    b_conv4_3 = bias_variable([512])
    output = tf.nn.relu(norm(conv2d(output, W_conv4_3) + b_conv4_3,norm_type='group',is_train = True,i='4_3'))
    print(output)

    W_conv4_4 = tf.get_variable('conv4_4', shape=[3, 3, 512, 512], initializer=tf.contrib.keras.initializers.he_normal())
    b_conv4_4 = bias_variable([512])
    output = tf.nn.relu(norm(conv2d(output, W_conv4_4) + b_conv4_4,norm_type='group',is_train = True,i='4_4'))
    output = max_pool(output, 2, 2)
    print(output)

    W_conv5_1 = tf.get_variable('conv5_1', shape=[3, 3, 512, 512], initializer=tf.contrib.keras.initializers.he_normal())
    b_conv5_1 = bias_variable([512])
    output = tf.nn.relu(norm(conv2d(output, W_conv5_1) + b_conv5_1,norm_type='group',is_train = True,i='5_1'))
    print(output)

    W_conv5_2 = tf.get_variable('conv5_2', shape=[3, 3, 512, 512], initializer=tf.contrib.keras.initializers.he_normal())
    b_conv5_2 = bias_variable([512])
    output = tf.nn.relu(norm(conv2d(output, W_conv5_2) + b_conv5_2,norm_type='group',is_train = True,i='5_2'))
    print(output)

    W_conv5_3 = tf.get_variable('conv5_3', shape=[3, 3, 512, 512], initializer=tf.contrib.keras.initializers.he_normal())
    b_conv5_3 = bias_variable([512])
    output = tf.nn.relu(norm(conv2d(output, W_conv5_3) + b_conv5_3,norm_type='group',is_train = True,i='5_3'))
    print(output)

    W_conv5_4 = tf.get_variable('conv5_4', shape=[3, 3, 512, 512], initializer=tf.contrib.keras.initializers.he_normal())
    b_conv5_4 = bias_variable([512])
    output = tf.nn.relu(norm(conv2d(output, W_conv5_4) + b_conv5_4,norm_type='group',is_train = True,i='5_4'))
    print(output)

    # output = tf.contrib.layers.flatten(output)
    output = tf.reshape(output, [-1, 2*2*512])
    print(output)

    W_fc1 = tf.get_variable('fc1', shape=[2048, 4096], initializer=tf.contrib.keras.initializers.he_normal())
    b_fc1 = bias_variable([4096])
    output = tf.nn.relu(norm(tf.matmul(output, W_fc1) + b_fc1,norm_type='batch',is_train = True,i='fc1') )
    output = tf.nn.dropout(output, keep_prob)
    print(output)

    W_fc2 = tf.get_variable('fc7', shape=[4096, 4096], initializer=tf.contrib.keras.initializers.he_normal())
    b_fc2 = bias_variable([4096])
    output = tf.nn.relu(norm(tf.matmul(output, W_fc2) + b_fc2,norm_type='batch',is_train = True,i='fc2'))
    output = tf.nn.dropout(output, keep_prob)
    print(output)

    W_fc3 = tf.get_variable('fc3', shape=[4096, 12], initializer=tf.contrib.keras.initializers.he_normal())
    b_fc3 = bias_variable([12])
    output = tf.nn.relu(norm(tf.matmul(output, W_fc3) + b_fc3,norm_type='batch',is_train = True,i='fc3'))
    print(output)
    # output  = tf.reshape(output,[-1,10])

    # loss function: cross_entropy
    # train_step: training operation
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=output))
    l2 = tf.add_n([tf.nn.l2_loss(var) for var in tf.trainable_variables()])
    train_step = tf.train.MomentumOptimizer(learning_rate, momentum_rate, use_nesterov=True).\
        minimize(cross_entropy + l2 * weight_decay)

    correct_prediction = tf.equal(tf.argmax(output, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    # initial an saver to save model
    saver = tf.train.Saver()

    with tf.Session() as sess:

        sess.run(tf.global_variables_initializer())
        summary_writer = tf.summary.FileWriter(log_save_path,sess.graph)

        # epoch = 164 
        # make sure [bath_size * iteration = data_set_number]

        for ep in range(1, total_epoch+1):
            lr = learning_rate_schedule(ep)
            pre_index = 0
            pre_index_test = 0
            train_acc = 0.0
            train_loss = 0.0
            start_time = time.time()

            print("\n epoch %d/%d:" % (ep, total_epoch))

            for it in range(1, iterations+1):
                batch_x = train_x[pre_index:pre_index+batch_size]
                batch_y = train_y[pre_index:pre_index+batch_size]
                
                batch_x = data_augmentation(batch_x)

                _, batch_loss = sess.run([train_step, cross_entropy],
                                         feed_dict={x: batch_x, y_: batch_y, keep_prob: dropout_rate,
                                                    learning_rate: lr, train_flag: True})
                batch_acc = accuracy.eval(feed_dict={x: batch_x, y_: batch_y, keep_prob: 1.0, train_flag: True})

                train_loss += batch_loss
                train_acc += batch_acc
                pre_index += batch_size

                if it == iterations:
                    
                    train_loss /= iterations
                    train_acc /= iterations

                    loss_, acc_ = sess.run([cross_entropy, accuracy],
                                           feed_dict={x: batch_x, y_: batch_y, keep_prob: 1.0, train_flag: True})
                    train_summary = tf.Summary(value=[tf.Summary.Value(tag="train_loss", simple_value=train_loss), 
                                               tf.Summary.Value(tag="train_accuracy", simple_value=train_acc)])

                    val_acc, val_loss, test_summary = run_testing(sess, ep)

                    summary_writer.add_summary(train_summary, ep)
                    summary_writer.add_summary(test_summary, ep)
                    summary_writer.flush()

                    print("iteration: %d/%d, cost_time: %ds, train_loss: %.4f, "
                          "train_acc: %.4f, test_loss: %.4f, test_acc: %.4f"
                          % (it, iterations, int(time.time()-start_time), train_loss, train_acc, val_loss, val_acc))
                else:
                    print("iteration: %d/%d, train_loss: %.4f, train_acc: %.4f"
                          % (it, iterations, train_loss / it, train_acc / it))

        save_path = saver.save(sess, model_save_path)
        print("Model saved in file: %s" % save_path) 

D:\Python\anaconda3.4\envs\py36\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


Q:/Github/GuanDong-AL-train/data/guangdong_round1_train2_20180916/无瑕疵样本/正常20180911102026.jpg------------is ok 
Q:/Github/GuanDong-AL-train/data/guangdong_round1_train2_20180916/无瑕疵样本/正常20180911102101.jpg------------is ok 
Q:/Github/GuanDong-AL-train/data/guangdong_round1_train2_20180916/无瑕疵样本/正常20180911102242.jpg------------is ok 
Q:/Github/GuanDong-AL-train/data/guangdong_round1_train2_20180916/无瑕疵样本/正常20180911104058对照样本.jpg------------is ok 
Q:/Github/GuanDong-AL-train/data/guangdong_round1_train2_20180916/无瑕疵样本/正常20180911104113对照样本.jpg------------is ok 
Q:/Github/GuanDong-AL-train/data/guangdong_round1_train2_20180916/无瑕疵样本/正常20180912103956对照样本.jpg------------is ok 
Q:/Github/GuanDong-AL-train/data/guangdong_round1_train2_20180916/无瑕疵样本/正常20180912104030对照样本.jpg------------is ok 
Q:/Github/GuanDong-AL-train/data/guangdong_round1_train2_20180916/无瑕疵样本/正常20180912140809对照样本.jpg------------is ok 
Q:/Github/GuanDong-AL-train/data/guangdong_round1_train2_20180916/无瑕疵样本/正常20180912140831对照样本